In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import random
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from matplotlib.image import imread



train_path ="panneaux\Dataset\Train"
test_path = "panneaux\Dataset\Test"


IMG_SIZE = (32, 32)


classes_number = len(os.listdir(train_path))
print(f"Nombre de classes: {classes_number}")


classes = { 
    0:'Speed limit (20km/h)',
    1:'Speed limit (30km/h)',
    2:'Speed limit (50km/h)',
    3:'Speed limit (60km/h)',
    4:'Speed limit (70km/h)',
    5:'Speed limit (80km/h)',
    6:'End of speed limit (80km/h)',
    7:'Speed limit (100km/h)',
    8:'Speed limit (120km/h)',
    9:'No passing',
    10:'No passing veh over 3.5 tons',
    11:'Right-of-way at intersection',
    12:'Priority road',
    13:'Yield',
    14:'Stop',
    15:'No vehicles',
    16:'Veh > 3.5 tons prohibited',
    17:'No entry',
    18:'General caution',
    19:'Dangerous curve left',
    20:'Dangerous curve right',
    21:'Double curve',
    22:'Bumpy road',
    23:'Slippery road',
    24:'Road narrows on the right',
    25:'Road work',
    26:'Traffic signals',
    27:'Pedestrians',
    28:'Children crossing',
    29:'Bicycles crossing',
    30:'Beware of ice/snow',
    31:'Wild animals crossing',
    32:'End speed + passing limits',
    33:'Turn right ahead',
    34:'Turn left ahead',
    35:'Ahead only',
    36:'Go straight or right',
    37:'Go straight or left',
    38:'Keep right',
    39:'Keep left',
    40:'Roundabout mandatory',
    41:'End of no passing',
    42:'End no passing veh > 3.5 tons' 
}


folders = os.listdir(train_path)

train_number = []
class_num = []

for folder in folders:
    train_files = os.listdir(train_path + '/' + folder)
    train_number.append(len(train_files))
    class_num.append(classes[int(folder)])

zipped_lists = zip(train_number, class_num)
sorted_pairs = sorted(zipped_lists)
tuples = zip(*sorted_pairs)
train_number, class_num = [list(tuple) for tuple in tuples]


plt.figure(figsize=(10, 7))
plt.barh(class_num, train_number)
plt.title("Distribution des classes dans le jeu d'entrainement")
plt.xlabel("Nombre d'images")
plt.ylabel("Classes")
plt.tight_layout()
plt.show()


def load_data(data_dir):
    images = []
    labels = []

    folders = sorted(os.listdir(data_dir))  
    for folder in folders:
        folder_path = os.path.join(data_dir, folder)
        if not os.path.isdir(folder_path):
            continue
        for image_file in os.listdir(folder_path):
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg')): 
                image_path = os.path.join(folder_path, image_file)
                image = cv2.imread(image_path)
                if image is None:
                    print("Impossible de lire:", image_path)
                    continue
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMG_SIZE)
                images.append(image)
                labels.append(int(folder))  
    return np.array(images), np.array(labels)

def load_test_data(test_dir, csv_file):
    df = pd.read_csv(csv_file)
    images = []
    labels = []

    for idx, row in df.iterrows():
        image_path = os.path.join(test_dir, os.path.basename(row['Path']))

        if not os.path.exists(image_path):
            print("Fichier non trouvé :", image_path)
            continue

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, IMG_SIZE)
        images.append(image)
        labels.append(row['ClassId'])

    return np.array(images), np.array(labels)







print("Chargement des donnees d'entraenement...")
X_train, y_train = load_data(train_path)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train 
)
print("Chargement des donnees de test...")
test_csv = r"D:\sara_projects\projets\panneaux\Dataset\Test.csv"
X_test, y_test = load_test_data(test_path, test_csv)

# Normalisation + categoriel conversion 
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Conversion des labels en one-hot
y_train_categorical = to_categorical(y_train, classes_number)
y_val_categorical = to_categorical(y_val, classes_number)
y_test_categorical = to_categorical(y_test, classes_number)




# Data augmentation

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.1,
    horizontal_flip=False, 
    fill_mode='nearest'
)

print("Data Augmentation configurée!")

In [ ]:
#CNN  model

def create_cnn_model():
    model = Sequential([
        # first 
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        BatchNormalization(),
        MaxPool2D(2, 2),
        Dropout(0.25),
        
        # 2 couche convolutive
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(2, 2),
        Dropout(0.25),
        
        # 3eme couche convolutive
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPool2D(2, 2),
        Dropout(0.25),
        
        # fully connected
        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(classes_number, activation='softmax')
    ])
    
    return model

model = create_cnn_model()

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# model trainning
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001
)

history = model.fit(
    datagen.flow(X_train, y_train_categorical, batch_size=32),
    epochs=50,
    validation_data=(X_val, y_val_categorical),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)



In [ ]:
history = model.fit(
    datagen.flow(X_train, y_train_categorical, batch_size=32),
    epochs=10,
    validation_data=(X_val, y_val_categorical),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

In [ ]:

#evaluation
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy du modèle')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss du modèle')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical, verbose=0)
print(f"Accuracy sur le jeu de test: {test_accuracy:.4f}")
print(f"Loss sur le jeu de test: {test_loss:.4f}")

# Matrice de confusion et rapport 

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred_classes)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=list(classes.values())[:10],  # Afficher seulement les 10 premières classes pour la lisibilité
            yticklabels=list(classes.values())[:10])
plt.title('Matrice de Confusion (10 premières classes)')
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Rapport de classification
print("Rapport de classification:")
print(classification_report(y_test, y_pred_classes, 
                          target_names=list(classes.values())))

def visualize_predictions(images, true_labels, pred_labels, classes_dict, num_samples=10):
    """
    Visualise quelques prédictions du modèle
    """
    indices = random.sample(range(len(images)), num_samples)
    
    plt.figure(figsize=(15, 8))
    for i, idx in enumerate(indices):
        plt.subplot(2, 5, i+1)
        plt.imshow(images[idx])
        
        true_label = classes_dict[true_labels[idx]]
        pred_label = classes_dict[pred_labels[idx]]
        
        color = 'green' if true_labels[idx] == pred_labels[idx] else 'red'
        plt.title(f'Vrai: {true_label}\nPrédit: {pred_label}', color=color, fontsize=8)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_predictions(X_test, y_test, y_pred_classes, classes)

model.save('traffic_sign_classifier.h5')


def predict_traffic_sign(image_path, model, classes_dict):
    
  
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, IMG_SIZE)
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    
    
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)
    
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
    plt.title(f'Prédiction: {classes_dict[predicted_class]}\nConfiance: {confidence:.2f}')
    plt.axis('off')
    plt.show()
    
    return classes_dict[predicted_class], confidence